# Semamtic Analysis

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import traceback
# import torch
import re
import random
import pandas as pd
# import swifter
# from transformers import pipeline
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /Users/allinone/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/allinone/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/allinone/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/allinone/nltk_data...


True

## Data Processing

The data here was downloaded from https://cseweb.ucsd.edu/~jmcauley/datasets/amazon_v2/

In [2]:
%%time


data = pd.read_json('Dataset\Movies_and_TV_5.json', lines=True)

CPU times: total: 33 s
Wall time: 35.8 s


In [3]:
data_temp = data

In [4]:
data = data_temp

Remove useless columns from the data and review text as empty

Please increase the N for more accurate predication

In [5]:
columns_to_drop = ['verified', 'reviewTime', 'style', 'reviewerName', 'vote', 'image', 'summary', 'unixReviewTime']
data = data.drop(columns=columns_to_drop)

data = data.dropna(subset=["rev iewText"])
data["reviewText"] = data["reviewText"].astype(str)

data = data.sample(n=10000)
data.reset_index(drop=True, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   overall     10000 non-null  int64 
 1   reviewerID  10000 non-null  object
 2   asin        10000 non-null  object
 3   reviewText  10000 non-null  object
dtypes: int64(1), object(3)
memory usage: 312.6+ KB


Perform filter out HTML tags and UTF-8 encoding symbols, tokenization, lowercasing, and lemmatization tasks on text data

In [6]:
def TextProcessing(text):
    # Remove HTML tag
    text = re.sub('<.*?>', '', text)

    # Remove special characters e.g \n, \xa0
    text = re.sub(r'\s+', ' ', text).strip()

    # Text cleaning
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    tokens = word_tokenize(text)
    
    # Lowercasing
    lowercased_tokens = [token.lower() for token in tokens]
    
    # Stopword removal
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in lowercased_tokens if token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    
    return lemmatized_tokens

Apply Text Processing on the reviews

In [7]:
%%time
data["reviewText"] = data["reviewText"].swifter.apply(TextProcessing)

Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

CPU times: total: 6.52 s
Wall time: 6.98 s


In [8]:
data = data[data['reviewText'].swifter.apply(lambda x: len(x) > 0)]

Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

In [9]:
data

,overall,reviewerID,asin,reviewText
0,5,A1UHWJON8XS24M,B00EF1PLV8,"[great, series]"
1,3,A3CO6TYS0K95H0,6301966554,"[still, great, movie, bluray, anything, specia..."
2,1,A7AG8Q6TH9G1A,B00005JPXY,"[total, waste, time, movie, made, kindergarten..."
3,5,AXOZ5BWOEDL76,B00TJFXH02,"[magician, astonishing, life, work, orson, wel..."
4,5,A3JETRFX9H6BMZ,B0051QS1Q8,"[love, interesting]"
...,...,...,...,...
9995,4,A26KD34XGDBG69,B00DY64A3U,"[si, fi, much, tip, good, one, could, written,..."
9996,5,ARQJ701ILWQO0,B00RI4VBUO,[unbelievable]
9997,5,A19K46GITRI5Y2,B00008DDVU,"[documentary, michael, moore, explores, source..."
9998,5,A2R2UQREX1QUH8,B00WEPI5WY,"[best, movie, church, history, early, persecut..."


## Sentiment analysis -- Model deployment 

Check CUDA avaliable on computer

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Executing the First semantic analysis model

In [14]:
pipe = pipeline("text-classification", model="LiYuan/amazon-review-sentiment-analysis", device = device)

In [15]:
def amazonreviewsentimentanalysis(text):
    text = "".join(text)
    # Example usage
    result = pipe(text)
    label = int(result[0]['label'].split()[0])
    score = label * result[0]['score']
    return score

In [16]:
%%time
data["Model A result"] = data["reviewText"].apply(amazonreviewsentimentanalysis)

C:\Users\hinlu\anaconda3\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


CPU times: total: 1min 20s
Wall time: 1min 33s


Executing the Second semantic analysis model

In [25]:
pipe = pipeline("text-classification", model="mmcquade11/reviews-sentiment-analysis", device=device)

In [28]:
def mmcquade11(text):
    text = "".join(text)
#     # Example usage
    result = pipe(text)
    label = result[0]['label']
    score = result[0]['score']

    if label == 'LABEL_1':
        score = 5*score  # Divide the score by 5
    elif label == 'LABEL_0':
        score = 1/score  # Divide the score by 1
    return(score)

In [29]:
%%time
data['model B result'] = data['reviewText'].apply(mmcquade11)

CPU times: total: 47.2 s
Wall time: 51.7 s


Executing the Third semantic analysis model

In [43]:
pipe = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")

In [44]:
def bertsentimentaluncased(text):
    text = "".join(text)
    # Example usage
    result = pipe(text)
    label = int(result[0]['label'].split()[0])
    score = label * result[0]['score']
    return(score)

In [45]:
%%time
data['model C result'] = data['reviewText'].apply(bertsentimentaluncased)

CPU times: total: 17min 52s
Wall time: 5min 15s


In [ ]:
data.to_json('smalltest.json', orient='records', lines=True)

In [46]:
data

,overall,reviewerID,asin,reviewText,Model A result,model C result,model B result
0,5,A1UHWJON8XS24M,B00EF1PLV8,"[great, series]",4.507430,3.638424,3.280221
1,3,A3CO6TYS0K95H0,6301966554,"[still, great, movie, bluray, anything, specia...",2.226870,0.435671,1.758164
2,1,A7AG8Q6TH9G1A,B00005JPXY,"[total, waste, time, movie, made, kindergarten...",3.706697,0.373875,1.749381
3,5,AXOZ5BWOEDL76,B00TJFXH02,"[magician, astonishing, life, work, orson, wel...",3.079194,1.542889,1.283880
4,5,A3JETRFX9H6BMZ,B0051QS1Q8,"[love, interesting]",3.846311,1.683720,3.478058
...,...,...,...,...,...,...,...
9995,4,A26KD34XGDBG69,B00DY64A3U,"[si, fi, much, tip, good, one, could, written,...",2.734165,0.265582,1.655769
9996,5,ARQJ701ILWQO0,B00RI4VBUO,[unbelievable],2.844120,2.267118,1.007441
9997,5,A19K46GITRI5Y2,B00008DDVU,"[documentary, michael, moore, explores, source...",3.079194,1.542889,1.283880
9998,5,A2R2UQREX1QUH8,B00WEPI5WY,"[best, movie, church, history, early, persecut...",4.561518,2.220508,3.385945


Storing data for ALS models, can be ingored

In [62]:
data.to_json('smalltest.json', orient='records', lines=True)

Calculate the average

In [ ]:
# Calculate the average and update the "overall" column
data['score'] = (data['overall'] + data['Model A result'] + data['model B result'] + data['model C result']) / 4

In [49]:
data

,overall,reviewerID,asin,reviewText,Model A result,model C result,model B result,score
0,5,A1UHWJON8XS24M,B00EF1PLV8,"[great, series]",4.507430,3.638424,3.280221,4.106519
1,3,A3CO6TYS0K95H0,6301966554,"[still, great, movie, bluray, anything, specia...",2.226870,0.435671,1.758164,1.855176
2,1,A7AG8Q6TH9G1A,B00005JPXY,"[total, waste, time, movie, made, kindergarten...",3.706697,0.373875,1.749381,1.707488
3,5,AXOZ5BWOEDL76,B00TJFXH02,"[magician, astonishing, life, work, orson, wel...",3.079194,1.542889,1.283880,2.726491
4,5,A3JETRFX9H6BMZ,B0051QS1Q8,"[love, interesting]",3.846311,1.683720,3.478058,3.502022
...,...,...,...,...,...,...,...,...
9995,4,A26KD34XGDBG69,B00DY64A3U,"[si, fi, much, tip, good, one, could, written,...",2.734165,0.265582,1.655769,2.163879
9996,5,ARQJ701ILWQO0,B00RI4VBUO,[unbelievable],2.844120,2.267118,1.007441,2.779670
9997,5,A19K46GITRI5Y2,B00008DDVU,"[documentary, michael, moore, explores, source...",3.079194,1.542889,1.283880,2.726491
9998,5,A2R2UQREX1QUH8,B00WEPI5WY,"[best, movie, church, history, early, persecut...",4.561518,2.220508,3.385945,3.791993


Keep only the columns needed to execute the ALS algorithm

In [ ]:
data = data.drop(columns=["reviewText", "Model A result", "model B result", "model C result", "overall"])

In [51]:
data

,reviewerID,asin,score
0,A1UHWJON8XS24M,B00EF1PLV8,4.106519
1,A3CO6TYS0K95H0,6301966554,1.855176
2,A7AG8Q6TH9G1A,B00005JPXY,1.707488
3,AXOZ5BWOEDL76,B00TJFXH02,2.726491
4,A3JETRFX9H6BMZ,B0051QS1Q8,3.502022
...,...,...,...
9995,A26KD34XGDBG69,B00DY64A3U,2.163879
9996,ARQJ701ILWQO0,B00RI4VBUO,2.779670
9997,A19K46GITRI5Y2,B00008DDVU,2.726491
9998,A2R2UQREX1QUH8,B00WEPI5WY,3.791993


In [52]:
duplicated_reviewerIDs = data.duplicated(subset='reviewerID', keep=False)
duplicated_reviewerIDs.sum()

1030

## Recommendation System

In [53]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Convert the Pandas DataFrame to PySpark DataFrame
df = spark.createDataFrame(data)

# Create StringIndexer transformers for reviewerID and asin columns
reviewerIDIndexer = StringIndexer(inputCol="reviewerID", outputCol="indexedReviewerID")
asinIndexer = StringIndexer(inputCol="asin", outputCol="indexedAsin")

# Fit StringIndexer on the DataFrame
indexerModel = reviewerIDIndexer.fit(df)
indexedDF = indexerModel.transform(df)
indexerModel = asinIndexer.fit(indexedDF)
indexedDF = indexerModel.transform(indexedDF)

indexedDF.show()

In [55]:
# Split data into 80% train, 20% test
training_data, test_data = indexedDF.randomSplit([0.8, 0.2], seed = 0)
training_data.show()
test_data.show()

+--------------+----------+------------------+-----------------+-----------+
|    reviewerID|      asin|             score|indexedReviewerID|indexedAsin|
+--------------+----------+------------------+-----------------+-----------+
|A101IGU6UDKW3X|6304406096| 2.726490834468174|            102.0|      412.0|
|A101L4HF0IZ33C|B00J9BZOXY|  2.46021622698448|            425.0|       96.0|
|A106YXO3EHVD3J|B002VECMB8| 2.726490834468174|            430.0|     4670.0|
|A10AVEAVBRW2LJ|B0009UZG2I| 2.726490834468174|            441.0|     3421.0|
| A10FPLE9JMYQG|B000GTJSO4|  4.44016695022583|            454.0|     3679.0|
|A10S6IJ9TV5BAU|B00144H00Y| 2.726490834468174|            477.0|     4163.0|
|A11BAS3GNT4A4M|B00S1ITA2W| 2.741661451756954|            503.0|       14.0|
|A11E9ON7T6LQO7|B000HEVTOU| 2.476490834468174|            514.0|     3696.0|
|A11EA10WV5RXR1|B000ZECQ08| 2.415416736202503|            516.0|      165.0|
|A11FOOTPCPZJHC|B000XA5K4I| 2.863236114382744|            520.0|      514.0|

In [56]:
from pyspark.ml.recommendation import ALS

als = ALS(maxIter=5,
          regParam=0.09,
          rank=25,
          userCol="indexedReviewerID",
          itemCol="indexedAsin",
          ratingCol="score",
          coldStartStrategy="drop",
          nonnegative=True)

model=als.fit(training_data)

In [57]:
predictions=model.transform(test_data)

evaluator=RegressionEvaluator(metricName="rmse",labelCol="score",predictionCol="prediction")

rmse=evaluator.evaluate(predictions)

print("RMSE="+str(rmse))

predictions.show()

RMSE=1.9578076895240408
+--------------+----------+------------------+-----------------+-----------+----------+
|    reviewerID|      asin|             score|indexedReviewerID|indexedAsin|prediction|
+--------------+----------+------------------+-----------------+-----------+----------+
|A3I8K4KC5GO2VT|B0000VJG72|4.1794463992118835|            312.0|      463.0| 1.2681159|
|A1TJIJKKN15KR9|B00006WUWP| 2.226490834468174|            169.0|      251.0|0.79596263|
|A11QW1O6PAYKQK|B00008MOUP| 2.990993095165383|            105.0|      458.0| 1.1909063|
|A2SGN1D92VK1X9|B005QI4U0W| 2.693380519747734|            247.0|       53.0| 2.2085085|
|A3KXJSZBDNLTXE|B00003CXS4|2.8184740021824837|             82.0|      918.0| 0.5895175|
|A1TA5QYECZP1L1|B00DJYJWVW| 2.726490834468174|             55.0|      322.0| 1.1970046|
|A2JSZ6OR4MRXD9|6303398375|2.9873037934303284|             64.0|      830.0|0.93176776|
|A3HKU4ZHFOY4NC|B00M4LP84Q| 2.226490834468174|            308.0|       26.0| 0.8046349|
| A3SS6V

In [1]:
user_recs=model.recommendForAllUsers(20).show(100)

NameError: name 'model' is not defined

In [61]:
recs=model.recommendForAllUsers(10).toPandas()

                reviewerID                                    recommendations
0     A0598193QWF1LXNNCV3O  [(B00GWXHYPI, 2.3959057331085205), (B005LAII0I...
1           A101541I8ZCSRU  [(B00000ILEA, 3.0313639640808105), (B01FL8ZD2A...
2           A10175AMUHOQC4  [(6305828075, 2.794443130493164), (B00006II6D,...
3           A101HW3MEASA23  [(6303049079, 2.8532235622406006), (B000NOIVT0...
4           A101IGU6UDKW3X  [(B004LWZWFQ, 2.762181282043457), (B0012Z367Q,...
...                    ...                                                ...
7516         AZXHK8IO25FL6  [(B0007KIFIC, 2.8829874992370605), (B00JBGF0SG...
7517         AZXJ8VJDRCWD2  [(6303039456, 3.2729949951171875), (B012DMPWUA...
7518         AZY1OM4M0YXXI  [(B01CEC0ECK, 2.4798243045806885), (6301170113...
7519         AZY6XD8GHT1F7  [(B0000YTP02, 2.3550803661346436), (B00GJNQ2NG...
7520         AZZ85I9JIWHVW  [(B00003CXKT, 2.413266181945801), (B002EWD0D6,...

[7521 rows x 2 columns]


C:\Users\hinlu\AppData\Local\Temp\ipykernel_30320\4252928346.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['recommendations'] = list(zip(new.asin, new.score))


In [ ]:
def display_All_recommendations(recs, indexedDF):
    # Extract recommendations into separate columns
    nrecs = recs.recommendations.apply(pd.Series) \
                .merge(recs, right_index=True, left_index=True) \
                .drop(["recommendations"], axis=1) \
                .melt(id_vars=['indexedReviewerID'], value_name="recommendation") \
                .drop("variable", axis=1) \
                .dropna()
    
    # Sort by indexedReviewerID
    nrecs = nrecs.sort_values('indexedReviewerID')
    
    # Map indexedReviewerID to reviewerID and indexedAsin to asin
    md = indexedDF.select(indexedDF['reviewerID'], indexedDF['indexedReviewerID'], indexedDF['asin'], indexedDF['indexedAsin'])
    md = md.toPandas()
    dict1 = dict(zip(md['indexedReviewerID'], md['reviewerID']))
    dict2 = dict(zip(md['indexedAsin'], md['asin']))
    nrecs['reviewerID'] = nrecs['UserID_index'].map(dict1)
    nrecs['asin'] = nrecs['indexedAsin'].map(dict2)
    
    # Reset index and select only relevant columns
    nrecs.reset_index(drop=True, inplace=True)
    new = nrecs[['reviewerID', 'asin', 'score']]
    
    # Group recommendations by reviewerID
    new['recommendations'] = list(zip(new.asin, new.score))
    res = new[['reviewerID', 'recommendations']]
    res_new = res['recommendations'].groupby([res.reviewerID]).apply(list).reset_index()
    
    return res_new

# Example usage:
recs = ...  # Your recommendation DataFrame
indexedDF = ...  # Your indexed DataFrame
output = display_recommendations(recs, indexedDF)
print(output)

In [ ]:
def display_recommendations_reviewerID(recs, indexedDF, reviewerID):
    # Extract recommendations into separate columns
    nrecs = recs.recommendations.apply(pd.Series) \
                .merge(recs, right_index=True, left_index=True) \
                .drop(["recommendations"], axis=1) \
                .melt(id_vars=['indexedReviewerID'], value_name="recommendation") \
                .drop("variable", axis=1) \
                .dropna()
    
    # Sort by indexedReviewerID
    nrecs = nrecs.sort_values('indexedReviewerID')
    
    # Map indexedReviewerID to reviewerID and indexedAsin to asin
    md = indexedDF.select(indexedDF['reviewerID'], indexedDF['indexedReviewerID'], indexedDF['asin'], indexedDF['indexedAsin'])
    md = md.toPandas()
    dict1 = dict(zip(md['indexedReviewerID'], md['reviewerID']))
    dict2 = dict(zip(md['indexedAsin'], md['asin']))
    nrecs['reviewerID'] = nrecs['UserID_index'].map(dict1)
    nrecs['asin'] = nrecs['indexedAsin'].map(dict2)
    
    # Filter recommendations for the given reviewerID
    nrecs = nrecs[nrecs['reviewerID'] == reviewerID]
    
    # Reset index and select only relevant columns
    nrecs.reset_index(drop=True, inplace=True)
    new = nrecs[['reviewerID', 'asin', 'score']]
    
    # Group recommendations by reviewerID
    new['recommendations'] = list(zip(new.asin, new.score))
    res = new[['reviewerID', 'recommendations']]
    res_new = res['recommendations'].groupby([res.reviewerID]).apply(list).reset_index()
    
    return res_new

Example for display_All_recommendations

In [ ]:
display_All_recommendations(recs, indexedDF)

Example for display_recommendations_reviewerID

In [ ]:
display_All_recommendations(recs, indexedDF, "A138DVM0KPI8MN")